In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
from sklearn import svm, preprocessing, pipeline
import sklearn.linear_model as LM
from sklearn.feature_selection import VarianceThreshold

import xgboost as xgb
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import cv
from scipy.stats import pearsonr
import lightgbm as lgb

# Model development below
Code is scattered, but the goal is to build a run_model function that takes in data, data type, and drug and returns single table comparing all models. Then the goal will be to iterate through data type combos and drugs and come up with a complete analysis output.

In [ ]:
from load_data import AMLData

In [ ]:
# point of access for all data
data = AMLData()

In [ ]:
en_model = LM.ElasticNet( 
    random_state=0,
    max_iter=100000, 
    fit_intercept=True,
    l1_ratio=.7,
    alpha=.9,
)

lasso_model = LM.Lasso(
    alpha=0.1,
    max_iter=100000, 
    fit_intercept=True,
)

svm_model = svm.SVR(C=1.0, epsilon=0.2, kernel='linear')
svc_model = svm.SVC(C=1.0, kernel='linear')

def run_sklearn(x_train, y_train, x_test, y_test, model, model_name, binarize=False):
        
    # don't think we need scaling? Can add in pretty quickly if we do
#     pipe = pipeline.Pipeline([
# #         ('scaler', preprocessing.StandardScaler()),
#         ('model', model)
#     ])
    if binarize:
        y_train, y_test = convert_to_binary(y_train, y_test)
    
    model.fit(x_train, y_train)
    train_pred = model.predict(x_train)
    preds = model.predict(x_test)
    error, r2, pearson = score_all(y_test, preds)
    coef = np.array(model.coef_).flatten()
    feature_names = model.feature_names_in_[coef>0]
    auc = np.nan
    if binarize:
        auc = metrics.average_precision_score(y_test, preds)
    return {
        'test_prediction': preds,
        'train_prediction': train_pred,
        'pearsonr': pearsonr(y_test, preds)[0],
        'mse': error,
        'r2' : r2,
        'model': model_name,
        'feature_names': feature_names,
        'auc': auc
    }


def convert_to_binary(y_train, y_test):
    """ Binarize AUC """
    y_train_c = np.copy(y_train)
    y_test_c = np.copy(y_test)
    y_train_c[y_train_c<100] = 1
    y_train_c[y_train_c>100] = 0
    y_test_c[y_test_c<100] = 1
    y_test_c[y_test_c>100] = 0
    return y_train_c, y_test_c


def run_gbt(x_train, y_train, x_test, y_test, feature_names, binarize=False):
    
    # again, don't know if we need to scale. Doesn't seem to have an impact...
#     scaler = preprocessing.StandardScaler()
#     x_train = scaler.fit_transform(x_train)
#     x_test = scaler.fit_transform(x_test)


    
    param = dict(
        device_type = 'cpu',
        boosting = 'gbdt',
        nthread = 1,
        objective = 'regression',
        metric = 'rmse',
        lambda_l1=1,
        lambda_l2=1,
        learning_rate = .01,
        tree_learner = 'serial',
        max_bin = 63,
        num_leaves = 10, 
        max_depth = 10,
        feature_fraction = .5,
        min_data_in_leaf = 1,
        min_gain_to_split = 1,
        verbose = -1
        
    )
    model_name = 'gbt'
    
    if binarize:
        param['objective'] = 'binary'
        param['metric'] = 'auc'
        y_train, y_test = convert_to_binary(y_train, y_test)
        model_name = 'gbt_binary'
        
    train_data = lgb.Dataset(x_train, label=y_train, feature_name=feature_names)
    validation_data = lgb.Dataset(x_test, label=y_test, feature_name=feature_names)
    num_round = 1000
    bst = lgb.train(
        param, 
        train_data, 
        num_round, 
        valid_sets=validation_data,
        callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=0)]
    )
    table = bst.feature_importance()
    feats = pd.Series(table, index=feature_names)
    selected_feat = feats[feats > 0].index.values
    
    train_pred = bst.predict(x_train, num_iteration=bst.best_iteration)
    preds = bst.predict(x_test, num_iteration=bst.best_iteration)
    error, r2, pearson = score_all(y_test, preds)
    auc = np.nan
    if binarize:
        auc = metrics.average_precision_score(y_test, preds)
    return {
        'test_prediction': preds,
        'train_prediction': train_pred,
        'mse': error,
        'r2' : r2,
        'pearsonr': pearsonr(y_test, preds)[0],
        'model': model_name,
        'feature_names': selected_feat,
        'auc': auc
    }
def score_all(y_test, preds):
    error = np.sqrt(metrics.mean_squared_error(y_test, preds))
    r2 = metrics.r2_score(y_test, preds)
    pearson = pearsonr(y_test, preds)[0]
#     print(f"RMSE: {error:0.3f} | R^2 {r2:0.3f} | R {pearson:0.3f}")
    return error, r2, pearson

In [ ]:

def run_model(data, d_sets, drug_name):
    df_subset = data.get_trainable_data(d_sets, drug_name)
    cols = list(set(df_subset.columns.values))
    cols.remove(drug_name)

    features = df_subset[cols].copy()
    target = df_subset[drug_name].values
    
    n_features_before = features.shape[1]
    
#     selector = VarianceThreshold(threshold=.001)
#     features = selector.fit_transform(features)
#     feature_names = selector.get_feature_names_out()
    
    features = features.loc[:, features.mean() > 0]
    features = features.loc[:, features.std() > 0]
    n_features = features.shape[1]
    print(f"Using {n_features} out of {n_features_before}"
          f" ({n_features_before - n_features} removed)")
    
#     features = pd.DataFrame(features, columns=feature_names)
    feature_names = list(set(features.columns.values))

    x_train, x_test, y_train, y_test = train_test_split(
        features,
        target,
        test_size=0.2,
        shuffle=True,
        random_state=101,
    )
    
    gbt_results = run_gbt(x_train, y_train, x_test, y_test, feature_names)
    gbt_binary_results = run_gbt(x_train, y_train, x_test, y_test, feature_names, binarize=True)
    enet_results = run_sklearn(x_train, y_train, x_test, y_test, en_model, 'EN')
    lasso_results = run_sklearn(x_train, y_train, x_test, y_test, lasso_model, 'LASSO')
    svm_results = run_sklearn(x_train, y_train, x_test, y_test, svm_model, 'SVM')
    svc_results = run_sklearn(x_train, y_train, x_test, y_test, svm_model, 'SVC', binarize=True)

    avg = np.zeros(len(enet_results['test_prediction']))
    for i in [gbt_results, enet_results, lasso_results, svm_results]:
        sns.regplot(y_test, i['test_prediction'], label=i['model'])
        avg += i['test_prediction']
    avg = avg/4
    score_all(y_test, avg)
    sns.regplot(y_test, avg, label='Average')
    plt.xlabel("Actual")
    plt.ylabel("Predicted")
    plt.legend()
    plt.suptitle(f"Drug: {drug_name}")
    results = pd.DataFrame(
        [gbt_results, gbt_binary_results, enet_results, lasso_results, svm_results, svc_results]
    )
    print(results[['model', 'mse', 'r2', 'pearsonr', 'auc']])
    return results
    

In [ ]:
m1 = run_model(
    data,
    ['wes',],
    'Venetoclax',
);

In [ ]:
m1

In [ ]:
m1 = run_model(
    data,
    ['rna_seq'],
    'Venetoclax',
);

In [ ]:
m1 = run_model(
    data,
    ['wes', 'phospho', 'proteomics', 'rna_seq'],
    'Venetoclax',
);

In [ ]:
x = set(m1['feature_names'].iloc[0])
y = set(m1['feature_names'].iloc[2])
z = set(m1['feature_names'].iloc[3])
hits = x.intersection(y).intersection(z)
for i in hits:
    print(i.split('-')[0].split('_')[0])

In [ ]:
m1 = run_model(
    data,
    ['wes', 'phospho', 'proteomics', 'rna_seq'],
    'Gilteritinib',
);

In [ ]:
m1 = run_model(
    data,   
    ['wes', 'phospho', 'proteomics', 'rna_seq'],
    'Quizartinib (AC220)',
);

In [ ]:
m1 = run_model(
    'Elesclomol',
);

# Test bed below
Started with xgboost but switched to lightGBM. It seems to be just as accurate but slightly faster. There is a Pytorch model below in the works. Most of below can be ignored for now.

In [ ]:
import lightgbm as lgb
def run_lgb(sources, drug_name, plot=False):
    df_subset = data.get_trainable_data(sources, drug_name)
    
    cols = list(set(df_subset.columns.values))
    cols.remove(drug_name)

    features = df_subset[cols].copy()
    target = df_subset[drug_name].values.reshape(-1, 1).ravel()
    
    n_features_before = features.shape[1]

    features = features.loc[:, features.mean() > 0]
    features = features.loc[:, features.std() > 0]
    feature_names = list(set(features.columns.values))
    n_features = features.shape[1]
    print(f"Using {n_features} out of {n_features_before}"
          f" ({n_features_before - n_features} removed)")
    
    X_train, X_test, y_train, y_test = train_test_split(
        features,
        target,
        test_size=0.2, 
        shuffle=True, 
        random_state=101,
    )
    
    scaler = preprocessing.StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    
    train_data = lgb.Dataset(X_train, label=y_train, feature_name=feature_names)
    validation_data = lgb.Dataset(X_test, label=y_test, feature_name=feature_names, )
    
#     validation_data = train_data.create_valid('validation.svm')
    
    param = dict(
        device_type='gpu',
        boosting='gbdt',
        nthread =16,
        objective='regression',
        metric='rmse',
#         lambda_l1=.5,
#         lambda_l2=.5,
        learning_rate= .01,
        tree_learner= 'serial',
        max_bin= 128,
        num_leaves= 20, 
        max_depth=6,
        feature_fraction= .5,
        min_data_in_leaf= 1,
        min_gain_to_split=1,
        verbose=0
        
    )

    
    num_round = 1000
    bst = lgb.train(
        param, 
        train_data, 
        num_round, 
        valid_sets=validation_data,
        callbacks=[lgb.early_stopping(stopping_rounds=100)]
    )
    bst.save_model('model.txt', num_iteration=bst.best_iteration)
#     lgb.plot_importance(bst, figsize =(4, 8))
#     plt.show()
    
    t_preds = bst.predict(X_train, num_iteration=bst.best_iteration)
    preds = bst.predict(X_test, num_iteration=bst.best_iteration)
    
    sns.regplot(y_train, t_preds, label='training')
    sns.regplot(y_test, preds, label='prediction')
    plt.legend()
    plt.show()
    error = np.sqrt(mean_squared_error(y_test, preds))
    r2 = r2_score(y_test, preds)
    print(f"Pearson correlation {pearsonr(y_test, preds)}")
    print(f"MSE: {error:0.3f} | $R^2$ {r2}")
    return bst
drug = 'Venetoclax'


In [ ]:
model = run_lgb('rna_seq', drug)
model = run_lgb('phospho', drug)
model = run_lgb('proteomics', drug)
model = run_lgb(['phospho', 'rna_seq', 'proteomics'], drug)

# Gradient boosted decision trees

XGBoost parameters


https://xgboost.readthedocs.io/en/latest/parameter.html

In [ ]:
# fit model no training data
params = dict(
    # general params
    nthread=1,
    booster='gbtree',
    gpu_id=0,
    seed=100,
    # regularization
    reg_alpha=.5, 
    reg_lambda=5,
    num_parallel_tree = 2, 
#     num_boost_round = 16,
    tree_method='gpu_hist',
    max_bin=128,
    objective='reg:squarederror',
    eval_metric='rmse',
    learning_rate=0.01,
    max_depth=5, 
    min_child_weight=1,
#     gamma=0,
    subsample=.5, # use half of data to resample
#     colsample_bytree=.8,
)
def create_importance_model(sources, drug_name, plot=False):
    df_subset = data.get_trainable_data(sources, drug_name)
    
    cols = list(set(df_subset.columns.values))
    cols.remove(drug_name)

    features = df_subset[cols].copy()
    target = df_subset[drug_name].values.reshape(-1, 1).ravel()
    
    n_features_before = features.shape[1]

    features = features.loc[:, features.mean() > 0]
    features = features.loc[:, features.std() > 0]
    feature_names = list(set(features.columns.values))
    n_features = features.shape[1]
    print(f"Using {n_features} out of {n_features_before}"
          f" ({n_features_before - n_features} removed)")
    
    X_train, X_test, y_train, y_test = train_test_split(
        features,
        target,
        test_size=0.2, 
        shuffle=True, 
        random_state=101,
    )
    scaler = preprocessing.StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    
    # organize data into xgb data matrix
    train = xgb.DMatrix(data=X_train, label=y_train)
    test = xgb.DMatrix(data=X_test, label=y_test)

    # add gene names as feature labels
    train.feature_names = feature_names
    test.feature_names = feature_names
    
    num_round = 10000
    results = dict()
    model = xgb.train(
        params, train, num_round,
        verbose_eval=100,
        early_stopping_rounds=100,
        evals=[(train, 'train'), (test, 'valid')],
        evals_result=results,
        
    )  
    
    feature_scores = model.get_fscore()
    s = pd.Series(list(feature_scores.values()), index=feature_scores)
    print(s.sort_values(ascending=False).head(5))
    
    #trained 
    t_preds = model.predict(train, iteration_range=(0, model.best_iteration + 1))

    #predictions
    preds = model.predict(test)
    error = np.sqrt(mean_squared_error(y_test, preds))
    r2 = r2_score(y_test, preds)
    print(f"MSE: {error:0.3f}")
    print(f"$R^2$ {r2}\n")
    if plot:
        # create plot of training and performance
        # plot training over time
        x_axis = range(0, len(results['train']['rmse']))
        plt.figure(figsize=(12, 6))
        plt.subplot(121)
        plt.plot(x_axis, results['train']['rmse'], label='Train')
        plt.plot(x_axis, results['valid']['rmse'], label='Test')
        plt.legend()

        # plot projections vs actual
        plt.subplot(122)
        sns.regplot(y_train, t_preds, label='training')
        sns.regplot(y_test, preds, label='prediction')
        plt.legend()
        plt.suptitle(f"{sources} {drug_name} : RMSE = {error}, $r^2$= {r2}")
    if not isinstance(sources, str):
        out_name = '_'.join(sorted(sources))
    else:
        out_name = sources
    return {
        'model': model,
        'data_sets': out_name,
        'drug_name': drug_name,
        'mse': error,
        'r2': r2
    }

In [ ]:
drug = 'Venetoclax'
model = create_importance_model('phospho', drug, True)['model']

In [ ]:
feature_scores = model.get_score(importance_type='total_gain')
feature_scores = pd.Series(list(feature_scores.values()), index=feature_scores)
feature_scores.sort_values(ascending=False)

In [ ]:
drug = 'Venetoclax'
model = create_importance_model('rna_seq', drug, True)['model']
feature_scores = model.get_score(importance_type='total_gain')
feature_scores = pd.Series(list(feature_scores.values()), index=feature_scores)
feature_scores.sort_values(ascending=False)

In [ ]:
feature_scores = model.get_score(importance_type='gain')
feature_scores = pd.Series(list(feature_scores.values()), index=feature_scores)
feature_scores.sort_values(ascending=False)

In [ ]:
drug = 'Quizartinib (AC220)'
create_importance_model('phospho', drug, True)
create_importance_model('proteomics', drug, True)
create_importance_model('rna_seq', drug, True)
create_importance_model(['phospho', 'rna_seq', 'proteomics'], drug, True)

In [ ]:
d_name = 'Venetoclax'
def run_input_sets(drug_name):
    output = []
    input_sets = [
        'rna_seq', 'proteomics', 'phospho',
#         ['rna_seq', 'proteomics'],
#         ['rna_seq', 'phospho'],
#         ['proteomics', 'phospho'],
#         ['rna_seq', 'proteomics', 'phospho'],
    ]
    for i in input_sets:
        result = create_importance_model(i, drug_name, True)
        del result['model']
        output.append(result)
    return output
def run_for_all():
    all_output= []
    for i in data.drug_names[:2]:
        x = run_input_sets(i)
        all_output.append(x)
    all_output = list(itertools.chain.from_iterable(all_output))
    return all_output
df = run_for_all()
df.to_csv('gbt_all_drugs_all_dtypes.csv')

In [ ]:
all_output

In [ ]:
all_output = pd.DataFrame(all_output)
# del all_output['model']
all_output

## K-fold 
K-fold implementation. 

In [ ]:
def run_kfold(sources, drug_name):
    df_subset = data.get_trainable_data(sources, drug_name)
    
    cols = list(set(df_subset.columns.values))
    cols.remove(drug_name)

    features = df_subset[cols].copy()
    target = df_subset[drug_name].values.reshape(-1, 1).ravel()
    
    n_features_before = features.shape[1]
    features = features.loc[:, features.mean() > 0]
    features = features.loc[:, features.std() > 0]
    feature_names = list(set(features.columns.values))
    n_features = features.shape[1]
    print(f"Using {n_features} out of {n_features_before}"
          f" ({n_features_before - n_features} removed)")
    
    # define data_dmatrix
    data_dmatrix = xgb.DMatrix(data=features, label=target)
    
    features = features.values
    
    # way 1
    features_output = []
    output_tracker = []
    scaler = preprocessing.StandardScaler()
    kf = KFold(n_splits=5, shuffle=True, random_state=101)
    for n, (train_index, test_index) in enumerate(kf.split(features)):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = target[train_index], target[test_index]

        X_train = scaler.fit_transform(X_train)
        X_test = scaler.fit_transform(X_test)

        # organize data into xgb data matrix
        train = xgb.DMatrix(data=X_train, label=y_train)
        test = xgb.DMatrix(data=X_test, label=y_test)

        # add gene names as feature labels
        train.feature_names = feature_names
        test.feature_names = feature_names
        
        num_round = 10000
        results = dict()
        model = xgb.train(
            params, train, num_round,
            verbose_eval=100,
            early_stopping_rounds=100,
            evals=[(train, 'train'), (test, 'valid')],
            evals_result=results,

        )  

        feature_scores = model.get_fscore()
        s = pd.Series(list(feature_scores.values()), index=feature_scores)
       
        s = s.to_frame().reset_index()
        s.rename(columns= {0: 'count'}, inplace=True)
        s.index.name = 'feature'
        s['nfold'] = n
        features_output.append(s)

        #trained 
        t_preds = model.predict(train, iteration_range=(0, model.best_iteration + 1))
        #predictions
        preds = model.predict(test)
        
        
        error = np.sqrt(mean_squared_error(y_test, preds))
        r2 = r2_score(y_test, preds)
        print(f"MSE: {error:0.3}")
        print(f"$R^2$ {r2}")
        output_tracker.append({'nfold':n, 'r2': r2, 'error':error})
    return pd.DataFrame(output_tracker), pd.concat(features_output, ignore_index=True)

#      way 2
#     kf = KFold(n_splits=5, shuffle=True, random_state=101)
#     for train_index, test_index in kf.split(features):
#         X_train, X_test = features[train_index], features[test_index]
#         y_train, y_test = target[train_index], target[test_index]
        
#         xgb_model = xgb.XGBRegressor(**params).fit(
#             X_train, y_train,
#             early_stopping_rounds=100,
#             eval_set=[(X_test, y_test)],
#             verbose=100,
            
#         )
        
#         print(xgb_model.feature_importances_)
#         predictions = xgb_model.predict(X_test)
#         print(mean_squared_error(y_test, predictions))
#      way 3
#     xgb_cv = cv(
#         dtrain=data_dmatrix,
#         params=params, 
#         nfold=5,
#         num_boost_round=50, 
#         early_stopping_rounds=10, 
#         metrics=["rmse"], 
#         as_pandas=True,
#         seed=123
#     )
#     return xgb_cv
    
    
stats, feat_out = run_kfold(['phospho', 'rna_seq', 'proteomics'], 'Venetoclax')

In [ ]:
total_selects = feat_out.groupby('index').sum()['count']
filtered = total_selects.sort_values(ascending=False).head(10)
sorted_index = list(filtered.index)
subset = feat_out.loc[feat_out['index'].isin(sorted_index)]

In [ ]:
sns.barplot(x='count', y='index', data=subset, orient='h', order=sorted_index);

In [ ]:
g = sns.catplot(x="count", y="index",
                hue="nfold", 
                data=subset, kind="bar", order=sorted_index,
                height=8, aspect=1);

In [ ]:
models = []
for i in ['Venetoclax', 'Gilteritinib', 'Quizartinib (AC220)',
              'Trametinib (GSK1120212)', 'Sorafenib']:
    models.append(create_importance_model('rna_seq', i, False))
    models.append(create_importance_model('proteomics', i, False))
    models.append(create_importance_model('phospho', i, False))
    models.append(
        create_importance_model(
            ['phospho', 'rna_seq', 'proteomics'], i, False
        )
    )
    

In [ ]:
df = pd.DataFrame(models, columns=['rmse', 'r2', 'data_type', 'drug'])
df.head()

In [ ]:
ax = sns.swarmplot(data=df, x="r2", y="drug", hue="data_type")
ax.set(ylabel="");
ax.set(xlabel="$R^2$");

In [ ]:
ex_plot_data = data.get_trainable_data(['rna_seq'], 'Sorafenib')

In [ ]:
ex_plot_data.head()

In [ ]:
sns.kdeplot(ex_plot_data['GMPPB_prot'], ex_plot_data['Sorafenib'])
sns.kdeplot(ex_plot_data['BCL9L_prot'], ex_plot_data['Sorafenib'])

# NN
Keeping for the moment...

In [ ]:
import torch
import torch.nn as nn
from tqdm.notebook import tqdm
class Net(nn.Module):
    def __init__(self, D_in, H1, D_out):
        super(Net, self).__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(D_in, H1,),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(H1, H1),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(H1, D_out),
        )

    def forward(self, x):
        y_pred = self.linear_stack(x)
        return y_pred
    

import time
from copy import deepcopy
def run_nn(save_name, drug_name, verbose=False):
    torch.cuda.empty_cache()
    dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    dtype = torch.float32
    
    df_subset = data.get_trainable_data(['phospho', 'proteomics', 'rna_seq'], drug_name)
    
    cols = list(set(df_subset.columns.values))
    cols.remove(drug_name)

    features = df_subset[cols].copy()
    target = df_subset[drug_name].values.reshape(-1, 1)
    n_features_before = features.shape[1]

    features = features.loc[:, features.mean() > 0]
    features = features.loc[:, features.std() > 0]
    n_features = features.shape[1]
    print(f"Using {n_features} out of {n_features_before}"
          f" ({n_features_before - n_features} removed)")
    
    try:
        X_train, X_test, y_train, y_test = train_test_split(
            features,
            target,
            test_size=0.2, 
            shuffle=True, 
            random_state=69,
        )
    except:
        print("error")
        return
    
    scaler = preprocessing.StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    
    
    X_train = torch.tensor(X_train, dtype=dtype,  device=dev)
    X_test = torch.tensor(X_test, dtype=dtype,  device=dev)

    y_train = torch.tensor(y_train, dtype=dtype,  device=dev)
    y_test = torch.tensor(y_test, dtype=dtype,  device=dev)

    D_in, D_out = X_train.shape[1], y_train.shape[1]
    H1 = int((D_in+D_out)/2)

    model = Net(D_in, H1, D_out)
    model.to(dev)
    model.to(dtype)
    
    criterion = nn.MSELoss(reduction='mean')
    
#     optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
    
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=1e-5,
        betas=(0.9, 0.999), 
        eps=1e-08, 
        weight_decay=0,
        amsgrad=False
    )
#     optimizer = torch.optim.SGD(
#         model.parameters(),
#         lr=0.001,
#         momentum=0.5
#     )
#     optimizer = torch.optim.ASGD(
#         model.parameters(), lr=1e-5, lambd=0.001, 
#         alpha=0.0, t0=1000.0, weight_decay=0.
#     )

    n_epoch = 10000
    eval_per_epoch = n_epoch//100
    prev_loss = np.inf
    steps_wo_impro = 0
    best_state = deepcopy(model.state_dict())
    best_epoch = 0
    st = time.time()
    train, valid = [], []
    p_bar = tqdm(range(n_epoch))
    for epoch in range(n_epoch):
        if steps_wo_impro > eval_per_epoch*10:
#             print(f"Early stopping at {epoch}")
            break
        model.train()
        # use original method
        optimizer.zero_grad()
        y_pred = model(X_train)
        loss = criterion(y_pred, y_train)
        if torch.isnan(loss):
            break        
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
        
            test_loss = criterion(model(X_test), y_test).item()

            train.append(loss.item())
            valid.append(test_loss)

            if test_loss < prev_loss:
                prev_loss = deepcopy(test_loss)
                steps_wo_impro = 0
                best_state = deepcopy(model.state_dict())
                best_epoch = epoch
            else:
                steps_wo_impro += 1
    #         if epoch < 1000:
    #             steps_wo_impro = 0
    #             prev_loss = np.inf
            if verbose:
                if (epoch+1) % eval_per_epoch == 0:
                    p_bar.set_description(
                        'Epoch [{}/{}], Loss train: {:2.4e}, test {:2.4e}'.format(
                            epoch+1, n_epoch, loss.item(), test_loss)
                    )
#     print(f'time : {time.time()-st}')
    del model
    
    test_model = Net(D_in, H1, D_out)
    test_model.load_state_dict(best_state)
    test_model.to(dtype)
    test_model.to(dev)
    test_model.eval()
    
    train_pred = test_model(X_train).cpu().detach().numpy()
    train_actual = y_train.cpu().detach().numpy()
    
    test_pred = test_model(X_test).cpu().detach().numpy()
    test_actual =  y_test.cpu().numpy()
    error = mean_squared_error(test_pred, test_actual)

    error = "{0:.3f}".format(np.sqrt(error))
    r2 = r2_score(np.array(test_actual).flatten(), 
                  np.array(test_pred).flatten())
    print(f"\t {error}")  
    print(f"r2 = {r2}")  
    
    # simple plot
    plt.figure(figsize=(12, 6))
    plt.subplot(121)
    x_axis = range(0, len(train))
    plt.plot(best_epoch, np.sqrt(prev_loss), marker='^')
    plt.plot(x_axis, np.sqrt(train), label='Train')
    plt.plot(x_axis, np.sqrt(valid), label='Test')
    plt.legend()

    plt.subplot(122)
    sns.regplot(train_pred[:,0], train_actual[:,0], label='Training')
    sns.regplot(test_pred[:,0], test_actual[:,0], label='Prediction')
    plt.legend()
    plt.suptitle(f"{drug_name} {save_name} : RMSE = {error} r^2 {r2:.2f}")
#     plt.close()
    return np.sqrt(mean_squared_error(test_pred, test_actual))

In [ ]:
run_nn('Venetoclax', 'Venetoclax', True) 